In [3]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

In [4]:
def calc_enmo(some_data):
    # Calculate the magnitue by first squaring all of the x, y, and z value, then summing them, and taking the square root.
    mag = ((some_data.applymap(lambda x : x**2)).sum(axis=1)).transform(lambda x : np.sqrt(x))
    # To calculate ENMO we subtract 1 (Gravity) from the vector magnitudes
    enmo = mag.transform(lambda x : x - 1)
    # Finally if we have any ENMO values less than 0 we round them up.
    enmo.loc[enmo.loc[:] < 0] = 0
    
    return mag, enmo

In [5]:
def calc_mad(some_data, device):
    some_data = some_data.dropna()
    all_mad = {}

    time_name = device + " Time"
    some_data[time_name] = some_data[time_name].apply(lambda x: x.replace(microsecond=0))
        
    # Grab the first timestamp from data
    # print(some_data.shape)
    start = some_data.loc[some_data.index[0], time_name]
    # Specify the amount of time to aggregate over
    agg_len = 5
    # Grab end of aggregation period
    end_time = start + timedelta(seconds=agg_len - 1)
    # Calculate the total length of the trial in seconds
    trial_length = (some_data.loc[some_data.index[-1], time_name] - start).total_seconds()
    # Runs the total length of trial divided by the length of time we aggregate over
    # essentialy creates a window of agg_len, and interval of agg_len
    for i in range(int(trial_length//agg_len)):
        # print(end_time)
        # Get agg_len seconds worth of accelerometer readings
        group_s = some_data.loc[(some_data[time_name] >= start) & (some_data[time_name] <= end_time), :]
        # print(group_s)
        # Get the mean X, Y, and Z of those readings
        agg_s = group_s.aggregate(lambda x : np.mean(x))
        # print(mean_s)
        # Calculate the mean accelerometer magnitude
        mag_s = agg_s[4]
        # print(f"{mag_s}")
        # Subtract the mean magnitude from each accelerometer magnitude from each vector magnitude and then take abs
        dif_mean = group_s[device + ' Magnitude'].apply(lambda x : abs(x - mag_s))
        # Caclulate the sum of all the vector mags - mean mags. Then divide by the number of vectors
        # print(dif_mean.sum())
        if device == "Actigraph":
            if dif_mean.shape[0] != 500 :
                print(f"Error {dif_mean.shape[0]} readings")
        mad = (dif_mean.sum()) / dif_mean.shape[0]
        # print(mad)

        # Add each Mad and the corresponding time to a list :
        all_mad[end_time] = mad
        # 
        start = end_time + timedelta(seconds=1)
        end_time = start + timedelta(seconds=agg_len - 1)

    mad_df = pd.Series(data=all_mad)
    return mad_df

In [35]:
# Define path of file
path = "C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\"
trial = input("Which Trial is being processed? ")
input_path = path + "Trial " + trial + "\\apple_aligned.csv"

Which Trial is being processed? 10e


In [36]:
# Creates a dictionary that contains the data of each trial, and its keys are each trial number.
temp = pd.read_csv(input_path)
temp["Actigraph Time"] = pd.to_datetime(temp["Actigraph Time"])
temp["Proxy Time"] = pd.to_datetime(temp["Proxy Time"])

# acti1, acti2, acti3, acti4, acti5 = temp.groupby("Actigraph ID")
acti1, acti2, acti3, acti4 = temp.groupby("Actigraph ID")
# data = [acti1[1], acti2[1], acti3[1], acti4[1], acti5[1]]
data = [acti1[1], acti2[1], acti3[1], acti4[1]]
print("Finished") 

Finished


In [37]:
# Calculate Magnitude and ENMO for each device in each trial.
pair_num = 0
for dev_pair in data :# Each trial has 5 device pairs.
    # Calculate magnitude and ENMO of actigraph
    # print(pair_num)
    acti_mag, acti_enmo = calc_enmo( dev_pair.loc[:, ["Actigraph X", "Actigraph Y", "Actigraph Z"]] )
    dev_pair.insert(10, "Actigraph Magnitude", acti_mag)
    dev_pair.insert(11, "Actigraph ENMO", acti_enmo)
    # Calculate magintude and ENMO of proxy
    proxy_mag, proxy_enmo = calc_enmo( dev_pair.loc[:, ["Proxy X", "Proxy Y", "Proxy Z"]] )
    dev_pair.insert(18, "Proxy Magnitude", proxy_mag)
    dev_pair.insert(19, "Proxy ENMO", proxy_enmo)
    dev_pair.loc[(dev_pair["Proxy X"].isna()), ["Proxy Magnitude", "Proxy ENMO"]] = np.nan
    if pair_num == 0 :
        trial_data = dev_pair
    elif pair_num < 3:
        trial_data = pd.concat([trial_data, dev_pair])
    else :
        trial_data = pd.concat([trial_data, dev_pair])
        trial_data.to_csv(path + "Trial " + trial + "//apple_aligned_v2.csv", index=False)
    pair_num += 1

In [38]:
trial_data

,Trial Number,Round Number,Speed,Actigraph ID,Actigraph Time,Order,Include,Actigraph X,Actigraph Y,Actigraph Z,Actigraph Magnitude,Actigraph ENMO,Proxy ID,Proxy Time,Reading #,Proxy X,Proxy Y,Proxy Z,Proxy Magnitude,Proxy ENMO
252000,10e,NaN,3.2 Hz,20,2022-05-31 15:06:50,252001,0,0.121,-0.332,-0.988,1.049290,0.049290,D09YVJVXDD,2022-05-31 15:06:50.000,NaN,0.393066,-0.739014,-0.961670,1.274932,0.274932
252001,10e,NaN,3.2 Hz,20,2022-05-31 15:06:50,252002,0,0.027,-0.316,-1.000,1.049088,0.049088,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
252002,10e,NaN,3.2 Hz,20,2022-05-31 15:06:50,252003,0,-0.031,-0.328,-1.012,1.064279,0.064279,D09YVJVXDD,2022-05-31 15:06:50.020,NaN,0.885498,-0.435547,-1.086426,1.467695,0.467695
252003,10e,NaN,3.2 Hz,20,2022-05-31 15:06:50,252004,0,-0.090,-0.348,-1.012,1.073940,0.073940,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
252004,10e,NaN,3.2 Hz,20,2022-05-31 15:06:50,252005,0,-0.137,-0.355,-0.992,1.062477,0.062477,D09YVJVXDD,2022-05-31 15:06:50.040,NaN,0.468018,0.208496,-1.034668,1.154577,0.154577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167995,10e,NaN,0.6 Hz,84,2022-05-31 15:20:49,167996,0,0.031,0.000,-1.027,1.027468,0.027468,D09YVJVXDD,2022-05-31 15:20:49.943,NaN,0.030029,-0.006836,-0.999756,1.000230,0.000230
167996,10e,NaN,0.6 Hz,84,2022-05-31 15:20:49,167997,0,0.031,0.000,-1.027,1.027468,0.027468,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
167997,10e,NaN,0.6 Hz,84,2022-05-31 15:20:49,167998,0,0.031,0.000,-1.027,1.027468,0.027468,D09YVJVXDD,2022-05-31 15:20:49.963,NaN,0.029785,-0.002197,-0.998535,0.998982,0.000000
167998,10e,NaN,0.6 Hz,84,2022-05-31 15:20:49,167999,0,0.031,0.000,-1.027,1.027468,0.027468,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
def aggregate_data(data, device):
    # Calculate the MAD for device
    device_mad = calc_mad(data.loc[:, [device + " Time", device + " X", device + " Y", device + " Z", 
                                               device + " Magnitude"]], device).rename(device + " MAD")
    # Intialize formula used to calculate RMS
    rms = lambda x : np.sqrt(np.mean(x**2))
    # Intialize functions to be used on data columns
    device_aggs = {"Order" : [np.min], device + " X": [rms], device + " Y": [rms], device + " Z": [rms], device + " Magnitude": [np.max],
                    device + " ENMO": [np.max]}
    # Group the actigraph data by second and perform aggregations
    if device == "Proxy":
        data[device + " Time"] = data[device + " Time"].apply(lambda x : x.replace(microsecond=0))
        device_rms = data.drop(columns=["Reading #"]).dropna().groupby(["Include", device + " Time"]).agg(device_aggs)
        # print(device_rms)
    else:
        device_rms = data.groupby(["Include", device + " Time"]).agg(device_aggs)
        # print(device_rms)
                            
    # print(device_rms)
    # Rename aggregate  columns
    device_rms= device_rms.reset_index()
    device_rms.columns = ["Include", device + " Time", "Order", device + " RMS X", device + " RMS Y", device + " RMS Z", 
                        device + " MAX MAGNITUDE", device + " MAX ENMO"]
    # Selecte middle minutes from eacht trial only
    device_rms = device_rms.loc[device_rms["Include"] == 1].drop(columns=["Include"])
    # print(device_rms)
    # Create a data frame that holds the meta trial data
    acti_meta = data.loc[:,["Trial Number", "Round Number", "Speed", device + " ID", "Order"]]
    #if device == "Proxy":
        #print(device_rms)
        #print(device_mad)
    # Merge the MAD calculation with the aggregated x, y, z, magintude, and ENMO
    device_mad = device_rms.merge(device_mad, how="left", left_on=device + " Time", right_on=device_mad.index)
    # Merge aggregated data with trial meta data
    device_sec = acti_meta.merge(device_mad, how='inner', on="Order")
    # print(device_sec.loc[55:65,:])
    return device_sec

In [41]:
# Calculate MAD for each trial, and also aggregate data to second level.
# Used to keep track of which device pair is being looked at
pair_num = 0
# Resets trial file
trial_final = None
# Iterate through each device pair (5 actigraphs  and 2 proxys per trial)
acti_num = 1
for acti_pair in data :
    print(f"Actigraph Number : {acti_num}")
    acti_pair["Round Number"] = 5 # Used for extra rounds
    # split the data by proxy ID
    
    # [proxy_1, proxy_2] = acti_pair.groupby("Proxy ID")
    # if acti_num == 1 :
        # print(f"Proxy 1: {proxy_1[0]} \nProxy 2: {proxy_2[0]}")
        # print(f"Proxy {acti_pair.iloc[1,6]}")
    acti_num += 1
    # print(proxy_1[1])

    # Aggreagte data and calculate mad for actigraph
    # acti_data = aggregate_data(acti_pair.iloc[:acti_pair.shape[0]//2, :], "Actigraph")
    acti_data = aggregate_data(acti_pair, "Actigraph")
    # print(acti_data)

    # Aggreagte data and calculate mad for proxy 1:
    # proxy1_data = aggregate_data(proxy_1[1], "Proxy")
    proxy1_data = aggregate_data(acti_pair, "Proxy")
    # print(proxy1_data)

    # Aggreagte data and calculate mad for proxy 2:
    # proxy2_data = aggregate_data(proxy_2[1], "Proxy")

    # Get rid of duplicate columns
    proxy1_data.drop(columns=["Trial Number", 'Round Number', 'Speed', 'Order'], inplace=True)
    # print(proxy1_data)
    # proxy2_data.drop(columns=["Trial Number", 'Round Number', 'Speed', 'Order'], inplace=True)
    # combine into one dataframe
    # temp_prox1 = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")

    # temp_prox2 = acti_data.merge(proxy2_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")


    if pair_num == 0 :
        # trial_final = pd.concat([temp_prox1, temp_prox2])
        trial_final = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")
    else:
        # temp = pd.concat([temp_prox1, temp_prox2])
        temp = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")
        trial_final = pd.concat([trial_final, temp])
        if pair_num == 3 :
            trial_final.to_csv(path + "Trial " + trial + "//apple_rms_v2.csv", index=False)
    pair_num += 1
trial_final

Actigraph Number : 1


C:\Users\Nick\AppData\Local\Temp/ipykernel_5092/124379684.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  some_data[time_name] = some_data[time_name].apply(lambda x: x.replace(microsecond=0))


Actigraph Number : 2
Actigraph Number : 3
Actigraph Number : 4


,Trial Number,Round Number,Speed,Actigraph ID,Order,Actigraph Time,Actigraph RMS X,Actigraph RMS Y,Actigraph RMS Z,Actigraph MAX MAGNITUDE,Actigraph MAX ENMO,Actigraph MAD,Proxy ID,Proxy Time,Proxy RMS X,Proxy RMS Y,Proxy RMS Z,Proxy MAX MAGNITUDE,Proxy MAX ENMO,Proxy MAD
0,10e,5,3.2 Hz,20,255001,2022-05-31 15:07:20,0.323941,0.354122,0.992218,1.257449,0.257449,NaN,D09YVJVXDD,2022-05-31 15:07:20,0.372516,0.345972,0.999059,1.410904,0.410904,NaN
1,10e,5,3.2 Hz,20,255101,2022-05-31 15:07:21,0.327758,0.370464,0.992242,1.288771,0.288771,NaN,D09YVJVXDD,2022-05-31 15:07:21,0.371773,0.336049,0.999628,1.385926,0.385926,NaN
2,10e,5,3.2 Hz,20,255201,2022-05-31 15:07:22,0.324342,0.359112,0.991931,1.275541,0.275541,NaN,D09YVJVXDD,2022-05-31 15:07:22,0.381779,0.340453,0.999082,1.368704,0.368704,NaN
3,10e,5,3.2 Hz,20,255301,2022-05-31 15:07:23,0.326767,0.370169,0.991602,1.279466,0.279466,NaN,D09YVJVXDD,2022-05-31 15:07:23,0.378310,0.325123,1.000938,1.398321,0.398321,NaN
4,10e,5,3.2 Hz,20,255401,2022-05-31 15:07:24,0.336031,0.354079,0.991266,1.268035,0.268035,3.261822e-02,D09YVJVXDD,2022-05-31 15:07:24,0.386235,0.340689,0.997833,1.400326,0.400326,0.060158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,10e,5,0.6 Hz,84,164501,2022-05-31 15:20:15,0.031000,0.000000,1.027000,1.027468,0.027468,NaN,D09YVJVXDD,2022-05-31 15:20:15,0.017458,0.015313,0.998117,1.002312,0.002312,NaN
416,10e,5,0.6 Hz,84,164601,2022-05-31 15:20:16,0.031000,0.000000,1.027000,1.027468,0.027468,NaN,D09YVJVXDD,2022-05-31 15:20:16,0.028339,0.014047,0.997884,1.003639,0.003639,NaN
417,10e,5,0.6 Hz,84,164701,2022-05-31 15:20:17,0.031000,0.000000,1.027000,1.027468,0.027468,NaN,D09YVJVXDD,2022-05-31 15:20:17,0.017742,0.010735,0.998005,1.001218,0.001218,NaN
418,10e,5,0.6 Hz,84,164801,2022-05-31 15:20:18,0.031000,0.000000,1.027000,1.027468,0.027468,NaN,D09YVJVXDD,2022-05-31 15:20:18,0.024779,0.016963,0.997981,1.003714,0.003714,NaN


In [183]:
# Combine all aligned
aligned_files = glob.glob(path + "Trial *//*aligned_V2.csv")
aligned_stack = None
for file in aligned_files:
    if aligned_stack is None:
        aligned_stack = pd.read_csv(file)
    else:
        temp = pd.read_csv(file)
        aligned_stack = pd.concat([aligned_stack, temp])
aligned_stack.to_csv(path + "//apple_aligned_mad_enmo.csv", index=False)

In [190]:
aggregated_files = glob.glob(path + "Trial *//*rms_v2.csv")
aggregated_stack = None
for file in aggregated_files:
    if aggregated_stack is None:
        aggregated_stack = pd.read_csv(file)
    else:
        temp = pd.read_csv(file)
        aggregated_stack = pd.concat([aggregated_stack, temp])
aggregated_stack.to_csv(path + "//apple_aggregated_mad_enmo.csv")

In [189]:
aggregated_files = glob.glob(path + "Trial *//*rms_v2.csv")
aggregated_files

['C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 01\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 02\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 03\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 04\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 05\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 06\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 06e\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 07\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 08\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 09\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial